CHANGELOG

v0.0
Convert Data_Color.Aug_v0.9 to rgb2gray version

In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import glob, numpy as np, tensorflow as tf, time, cv2
from tensorflow.keras import layers
from IPython.display import HTML, Javascript, display

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset, shard_dataset

[ 2024-01-07 00:00:06 ] CUDA_VISIBLE_DEVICES automatically set to: 0           


In [2]:
root = '/home/jjlou/Jerry/wsi-arterio/vessel_detection_and_rough_segmentation/data_test'

In [3]:
# random flip via tensorflow layers
def aug(images, mask):
    
    augment = tf.keras.Sequential([
        layers.RandomTranslation(0.05, 0.05),
        layers.RandomRotation(1),
        layers.RandomFlip(),
    ])
       
    images = tf.cast(images, 'uint8')
    mask = tf.cast(mask, 'uint8') 
    
    mask = tf.stack([mask, mask, mask], -1)
    
    images_mask = tf.concat([images, mask], -1)  
    images_mask = augment(images_mask)  
    
    image = images_mask[:,:,:3]
    mask = images_mask[:,:,3]
    mask = tf.expand_dims(mask,axis=2)
    
    return tf.cast(image, 'uint8'), tf.cast(mask, 'uint8') 

In [4]:
def aug_rgb2gray(image, mask):
    image = np.reshape(image, (512,512,3))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = tf.stack([image, image, image], -1)
    return tf.cast(image, 'uint8'), tf.cast(mask, 'uint8')

In [5]:
#tf.numpy_function creates output with unknown shape.. so need to reshape it
def reshape(image, mask):
    image = tf.reshape(image, [512,512,3])
    mask = tf.reshape(mask, [512,512])
    return tf.cast(image, 'uint8'), tf.cast(mask, 'uint8')

In [6]:
load = sorted(glob.glob(f'{root}/train_Basic.Aug/*/*'))
save = f'{root}/train_Gray.Basic.Aug_sharded'

In [7]:
for s in load:
    ID = s.split('/')[-2]
    num_shard = s.split('/')[-1]
    instance_num = 0
    block = tf.data.Dataset.load(s)
    block_length = len(block)
    if not glob.glob(f'{save}/{ID}/{num_shard}_instance.{block_length-1}/*/*/*.snapshot'):
        for i, l in block:
            shard_save = f'{save}/{ID}/{num_shard}_instance.{instance_num}'
            if not glob.glob(f'{shard_save}/*/*/*.snapshot'):
                shard = tf.data.Dataset.from_tensors((i,l))
                shard = (
                    shard.map(lambda i,m: reshape(i,m), num_parallel_calls=tf.data.AUTOTUNE)
                    .map(lambda i,m: tf.numpy_function(aug_rgb2gray, [i,m], [tf.uint8, tf.uint8]), num_parallel_calls=tf.data.AUTOTUNE)
                    .map(lambda i,m: reshape(i,m), num_parallel_calls=tf.data.AUTOTUNE)
                    .map(lambda i,m: aug(i,m), num_parallel_calls=tf.data.AUTOTUNE)
                    .batch(1)
                )
                shard.save(shard_save)
            else:
                instance_num += 1
        restart_kernel()
        time.sleep(10)
    else:
        continue